<a href="https://colab.research.google.com/github/mauricio-ms/motor-imagery-convolutional-recurrent-neural-network/blob/master/notebooks/cnn-1d-tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


Mount Google Drive directory

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Install needed packages

In [3]:
!pip install colorlog

In [4]:
import logging
import colorlog


def get_logger(name="__main__", debug=False):
    log_format = (
        "[%(levelname)s] "
        "%(asctime)s - "
        "%(name)s: "
        "%(funcName)s - "
        "%(message)s"
    )
    bold_seq = "\033[1m"
    colorlog_format = (
        f"{bold_seq} "
        "%(log_color)s "
        f"{log_format}"
    )
    colorlog.basicConfig(format=colorlog_format)

    logger = logging.getLogger(name)
    if debug:
        logger.setLevel(logging.DEBUG)
    else:
        logger.setLevel(logging.INFO)

    # Output full log
    fh = logging.FileHandler("app.log")
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter(log_format)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    return logger

In [9]:
import os
import numpy as np

LOGGER = get_logger("physionet.py")
N_CHANNELS = 64
FEATURE_DESCRIPTION = {
    "X": tf.io.FixedLenFeature([N_CHANNELS], tf.float32),
    "y": tf.io.FixedLenFeature([], tf.int64)
}
TFRECORD_FILES_DIR = "drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/preprocessed-tfrecord-files"

In [14]:
def load_data(train_size=0.75, validation_size=None, n_subjects=None, **kwargs):
    LOGGER.info("Loading Physionet dataset ...")
    subjects = np.array(sorted(os.listdir(TFRECORD_FILES_DIR)))
    if n_subjects is not None:
        np.random.shuffle(subjects)
        subjects = subjects[:n_subjects]
    train_subjects, test_subjects = _train_test_split_subjects(subjects, train_size)
    if validation_size is not None:
        train_subjects, validation_subjects = _train_test_split_subjects(train_subjects, 1-validation_size)
        LOGGER.info(f"(Train, Validation, Test) Subjects = "
                    f"({len(train_subjects)}, {len(validation_subjects)}, {len(test_subjects)})")
        return _load_set(train_subjects, **kwargs), \
            _load_set(validation_subjects, **kwargs), \
            _load_set(test_subjects, **kwargs)

    LOGGER.info(f"(Train, Test) Subjects = ({len(train_subjects)}, {len(test_subjects)})")
    return _load_set(train_subjects, **kwargs), _load_set(test_subjects, **kwargs)


def _train_test_split_subjects(subjects, train_size):
    train_subjects_mask = np.random.rand(len(subjects)) < train_size
    return subjects[train_subjects_mask], subjects[~train_subjects_mask]


# TODO - convert_to_2d should define the data directory
def _load_set(subjects, n_readers=8, n_parse_threads=8, batch_size=100,
              convert_to_2d=False, expand_dim=False):
    path_files = [os.path.join(TFRECORD_FILES_DIR, subject, file_name)
                  for subject in subjects
                  for file_name in sorted(os.listdir(os.path.join(TFRECORD_FILES_DIR, subject)))]
    dataset = tf.data.Dataset.list_files(path_files)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TFRecordDataset(path_files, compression_type="GZIP"),
        cycle_length=n_readers)
    dataset = dataset.batch(batch_size).prefetch(1)
    dataset = dataset.map(lambda r: _preprocess(r, expand_dim=expand_dim),
                          num_parallel_calls=n_parse_threads)
    # dataset = dataset.cache()
    return dataset


@tf.function
def _preprocess(serialized_eeg_records, expand_dim=False):
    parsed_eeg_records = tf.io.parse_example(serialized_eeg_records, FEATURE_DESCRIPTION)
    X = parsed_eeg_records["X"]
    y = parsed_eeg_records["y"]
    if expand_dim:
        X = X[..., np.newaxis]
    return X, y

In [15]:
train_set, test_set = load_data(expand_dim=True)

  [INFO] 2020-08-17 00:40:39,053 - physionet.py: load_data - Loading Physionet dataset ...
  [INFO] 2020-08-17 00:40:39,057 - physionet.py: load_data - (Train, Test) Subjects = (81, 27)


UnimplementedError: ignored

In [12]:
from tensorflow import keras

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)
print("REPLICAS: ", strategy.num_replicas_in_sync)

with strategy.scope():
  model = keras.models.Sequential([
      keras.layers.Conv1D(32, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME",
                          input_shape=[64, 1]),
      keras.layers.Conv1D(64, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Conv1D(128, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1024, activation="relu",
                        kernel_initializer="he_normal"),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(5, activation="softmax")
  ])

  optimizer = keras.optimizers.Adam(lr=1e-4)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])

Running on TPU  ['10.119.1.98:8470']


  [WARNING] 2020-08-17 00:35:25,338 - tensorflow: initialize_tpu_system - TPU system grpc://10.119.1.98:8470 has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.


INFO:tensorflow:Initializing the TPU system: grpc://10.119.1.98:8470


  [INFO] 2020-08-17 00:35:25,339 - tensorflow: initialize_tpu_system - Initializing the TPU system: grpc://10.119.1.98:8470


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


  [INFO] 2020-08-17 00:35:41,506 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


  [INFO] 2020-08-17 00:35:41,507 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [13]:
LOGGER.info("Starting training CNN 1D model ...")
model.fit(train_set, epochs=2)
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-17 00:36:06,117 - physionet.py: <module> - Starting training CNN 1D model ...


InvalidArgumentError: ignored

In [ ]:
train_set

<PrefetchDataset shapes: ((None, 5, 1), (None, 1)), types: (tf.float32, tf.int32)>